In [2]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

sess = sagemaker.Session()
bucket='sagemaker-roadinspection-data' 
print(bucket)
prefix = 'v4'

arn:aws:iam::147962123592:role/service-role/AmazonSageMaker-ExecutionRole-20200623T163146
sagemaker-roadinspection-data
CPU times: user 111 ms, sys: 12.3 ms, total: 123 ms
Wall time: 176 ms


In [3]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print (training_image)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


544295431143.dkr.ecr.ap-southeast-2.amazonaws.com/image-classification:latest


In [4]:
s3train = 's3://{}/{}/Train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/Validation/'.format(bucket, prefix)
s3train_lst = 's3://{}/{}/train_lst/'.format(bucket, prefix)
s3validation_lst = 's3://{}/{}/validation_lst/'.format(bucket, prefix)
print(s3train)
print(s3validation)
print(s3train_lst)
print(s3validation_lst)

s3://sagemaker-roadinspection-data/v4/Train/
s3://sagemaker-roadinspection-data/v4/Validation/
s3://sagemaker-roadinspection-data/v4/train_lst/
s3://sagemaker-roadinspection-data/v4/validation_lst/


In [19]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
ic = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [27]:
ic.set_hyperparameters(num_layers=50,
                             use_pretrained_model=0,
                             image_shape = "3,224,224",
                             num_classes=2,
                             mini_batch_size=64,
                             epochs=15,
                             learning_rate=0.01,
                             top_k=2,
                             num_training_samples=4584,
                             #resize = 256,
                             precision_dtype='float32')

In [28]:
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
train_data_lst = sagemaker.session.s3_input(s3train_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data_lst = sagemaker.session.s3_input(s3validation_lst, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data, 
                 'train_lst': train_data_lst, 'validation_lst': validation_data_lst}

In [29]:
ic.fit(inputs=data_channels, logs=True)

2020-07-10 05:57:36 Starting - Starting the training job...
2020-07-10 05:57:38 Starting - Launching requested ML instances.........
2020-07-10 05:59:11 Starting - Preparing the instances for training......
2020-07-10 06:00:18 Downloading - Downloading input data......
2020-07-10 06:01:19 Training - Downloading the training image..Docker entrypoint called with argument(s): train
[07/10/2020 06:01:45 INFO 140628876748608] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[07/10/2020 06:01:45 INFO 140628876748608] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {u'